In [1]:
import ads
import oci
import requests
import tempfile

from ads.model.framework.sklearn_model import SklearnModel
from oci.signer import Signer
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

ads.set_auth(auth="resource_principal")

# Load dataset and Prepare train and test split
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Train a LogisticRegression model
sklearn_estimator = LogisticRegression()
sklearn_estimator.fit(X_train, y_train)

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



LogisticRegression()

In [2]:
# Instantiate ads.model.framework.sklearn_model.SklearnModel using the sklearn LogisticRegression model
sklearn_model = SklearnModel(
    estimator=sklearn_estimator, artifact_dir=tempfile.mkdtemp()
)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [3]:
# Autogenerate score.py, serialized model, runtime.yaml, input_schema.json and output_schema.json
#sklearn_model.prepare(
#    inference_conda_env="dbexp_p38_cpu_v1",
#    X_sample=X_train,
#    y_sample=y_train,
#)

sklearn_model.prepare(
    inference_conda_env="automlx251_p311_cpu_x86_64_v2",
    X_sample=X_train,
    y_sample=y_train,
    force_overwrite=True,
)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

  warnings.warn("slug will be deprecated. Provide conda pack path instead.")



Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [4]:
# Verify generated artifacts
sklearn_model.verify(X_test)

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

Start loading model.joblib from model directory /tmp/tmp_v7qjvzb ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [5]:
# Register scikit-learn model
model_id = sklearn_model.save(display_name="DIY SKLearn Model For Iris Classification")

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

Start loading model.joblib from model directory /tmp/tmp_v7qjvzb ...
Model is successfully loaded.
['output_schema.json', 'score.py', 'runtime.yaml', 'model.joblib', '.model-ignore', 'input_schema.json']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [6]:
# Deploy and create an endpoint for the SKLearn model
sklearn_model.deploy(
    display_name="DIY SKLearn Model For Iris Classification",
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {sklearn_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
sklearn_model.summary_status()

Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqianieosnwiryr54hydeke3saccn3p2gnixgcfrir56af4q


Creating model deployment:   0%|          | [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqianieosnwiryr54hydeke3saccn3p2gnixgcfrir56af4q


Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [9]:
# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.cloud.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm
config = oci.config.from_file("~/.oci/config") # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = sklearn_model.model_deployment.url + '/predict'
body = {"data": [[5.1, 3.5, 1.4, 0.2], [5.0, 2.0, 3.5, 1.0], [6.3, 3.3, 6.0, 2.5]]} # payload goes here
headers = {} # header goes here

print("Response from deployed model follows...")
requests.post(endpoint, json=body, auth=auth, headers=headers).json()

Response from deployed model follows...


{'prediction': [0, 1, 2]}

In [10]:
# Alternately, the OCI CLI may be invoked with the command line
print("Use the following command line to invoke the deployment via the OCI CLI...\n")
print("oci raw-request --http-method POST --target-uri " + sklearn_model.model_deployment.url + "/predict --request-body '{\"data\": [[5.1, 3.5, 1.4, 0.2], [5.0, 2.0, 3.5, 1.0], [6.3, 3.3, 6.0, 2.5]]}'")

Use the following command line to invoke the deployment via the OCI CLI...

oci raw-request --http-method POST --target-uri https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqianieosnwiryr54hydeke3saccn3p2gnixgcfrir56af4q/predict --request-body '{"data": [[5.1, 3.5, 1.4, 0.2], [5.0, 2.0, 3.5, 1.0], [6.3, 3.3, 6.0, 2.5]]}'
